In [7]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Data Generation
===

In [27]:
np.random.seed(10)
p, q = (np.random.rand(i, 2) for i in (4, 5))
p_big, q_big = (np.random.rand(i, 80) for i in (100, 120))

#print(p, "\n\n", q)
print (p_big)

[[0.91777412 0.71457578 0.54254437 ... 0.49504863 0.97708073 0.44077382]
 [0.31827281 0.51979699 0.57813643 ... 0.02116519 0.75046461 0.17604213]
 [0.45851421 0.51312271 0.48402089 ... 0.28825314 0.27479513 0.60105189]
 ...
 [0.09223334 0.45285232 0.94842416 ... 0.06463297 0.53147919 0.51179774]
 [0.4444723  0.9559604  0.57988831 ... 0.49981937 0.27882109 0.97201969]
 [0.37862344 0.24592521 0.76662351 ... 0.51365543 0.81850517 0.56299811]]


Solution
===

In [13]:
def naive(p, q):
    d = np.zeros((p.shape[0], q.shape[0]))
    for i in range(p.shape[0]):
        for j in range(q.shape[0]):
            tmp = 0
            for k in range(p.shape[1]):
                tmp += (p[i,k]-q[j,k])**2
            d[i,j] = tmp
    return np.sqrt(d)


In [15]:
q.shape[0]
q.shape[1]

2

### Use matching indices

Instead of iterating through indices, one can use them directly to parallelize the operations with Numpy.

In [22]:
rows, cols = np.indices((p.shape[0], q.shape[0]))  #np.indices((5,5))
print(rows, end='\n\n')
print(cols)

[[0 0 0 0 0]
 [1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]]

[[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]


In [17]:
print(p[rows.ravel()], end='\n\n')
print(q[cols.ravel()])

[[0.77132064 0.02075195]
 [0.77132064 0.02075195]
 [0.77132064 0.02075195]
 [0.77132064 0.02075195]
 [0.77132064 0.02075195]
 [0.63364823 0.74880388]
 [0.63364823 0.74880388]
 [0.63364823 0.74880388]
 [0.63364823 0.74880388]
 [0.63364823 0.74880388]
 [0.49850701 0.22479665]
 [0.49850701 0.22479665]
 [0.49850701 0.22479665]
 [0.49850701 0.22479665]
 [0.49850701 0.22479665]
 [0.19806286 0.76053071]
 [0.19806286 0.76053071]
 [0.19806286 0.76053071]
 [0.19806286 0.76053071]
 [0.19806286 0.76053071]]

[[0.16911084 0.08833981]
 [0.68535982 0.95339335]
 [0.00394827 0.51219226]
 [0.81262096 0.61252607]
 [0.72175532 0.29187607]
 [0.16911084 0.08833981]
 [0.68535982 0.95339335]
 [0.00394827 0.51219226]
 [0.81262096 0.61252607]
 [0.72175532 0.29187607]
 [0.16911084 0.08833981]
 [0.68535982 0.95339335]
 [0.00394827 0.51219226]
 [0.81262096 0.61252607]
 [0.72175532 0.29187607]
 [0.16911084 0.08833981]
 [0.68535982 0.95339335]
 [0.00394827 0.51219226]
 [0.81262096 0.61252607]
 [0.72175532 0.29187607

In [18]:
def with_indices(p, q):
    rows, cols = np.indices((p.shape[0], q.shape[0]))
    distances = np.sqrt(np.sum((p[rows, :] - q[cols, :])**2, axis=2))
    return distances

### Use a library

`scipy` is the equivalent of matlab toolboxes and have a lot to offer. Actually the pairwise computation is part of the library through the `spatial` module.

In [29]:
from scipy.spatial.distance import cdist

def scipy_version(p, q):
    return cdist(p, q)

### Numpy Magic

In [ ]:
def tensor_broadcasting(p, q):
    return np.sqrt(np.sum((p[:,np.newaxis,:]-q[np.newaxis,:,:])**2, axis=2))

# Compare methods

In [ ]:
methods = [naive, with_indices, scipy_version, tensor_broadcasting]
timers = []
for f in methods:
    r = %timeit -o f(p_big, q_big)
    timers.append(r)

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(np.arange(len(methods)), [r.best*1000 for r in timers], log=False)  # Set log to True for logarithmic scale
plt.xticks(np.arange(len(methods))+0.2, [f.__name__ for f in methods], rotation=30)
plt.xlabel('Method')
plt.ylabel('Time (ms)')
plt.show()

In [28]:
naive(p_big, q_big)

array([[3.5463562 , 3.565697  , 3.49523531, ..., 3.45806434, 3.45665815,
        3.81388442],
       [4.00188263, 3.53214057, 3.27800283, ..., 3.71310672, 3.95344582,
        3.59536741],
       [4.17522292, 3.56879999, 3.41938894, ..., 4.02732215, 3.92961253,
        3.58403141],
       ...,
       [3.62521939, 3.45146476, 3.80998935, ..., 4.28665811, 3.94376699,
        3.57750727],
       [3.74235978, 3.66972067, 3.58013781, ..., 3.84275737, 3.61753849,
        3.699406  ],
       [3.94677398, 3.3584521 , 3.90542048, ..., 3.52062854, 4.00497389,
        3.44475958]])